As we have seen in previous sections, hvPlot bakes in interactivity by automatically creating widgets when using ``groupby``. These widgets can be refined using [Panel](https://panel.pyviz.org). Panel allows you to customize the interactivity of your hvPlot output and provides more fine-grained control over the layout.

<div class="alert alert-warning" role="alert">
  When viewing on a static website, the widgets will be inoperable. To explore this functionality fully, download the notebook and run it!
</div>

In [ ]:
import panel as pn
import hvplot.pandas  # noqa

from bokeh.sampledata.iris import flowers

When ``groupby`` is used, the default widget is selected for the data type of the column. In this case since 'species' is composed of strings, the widget an instance of the class ``pn.widgets.Select``.

In [ ]:
flowers.hvplot.bivariate(x='sepal_width', y='sepal_length', width=600, 
                         groupby='species')

### Customizing Widgets

We can change where the widget is shown using the ``widget_location`` option.

In [ ]:
flowers.hvplot.bivariate(x='sepal_width', y='sepal_length', width=600, 
                         groupby='species', widget_location='left_top')

We can also change what the class of the widget is, using the ``widgets`` dict. For instance if we want to use a slider instead of a selector we can specify that.

In [ ]:
flowers.hvplot.bivariate(x='sepal_width', y='sepal_length', width=600, 
                         groupby='species', widgets={'species': pn.widgets.DiscreteSlider})

### Using widgets as arguments

So far we have only been dealing with widgets that are produced when using the ``groupby`` key word. But panel provides many other ways of expanding the interactivity of hvplot objects. For instance we might want to allow the user to select which fields to plot on the ``x`` and ``y`` axes. Or even what ``kind`` of plot to produce.

In [ ]:
x = pn.widgets.Select(name='x', options=['sepal_width', 'petal_width'])
y = pn.widgets.Select(name='y', options=['sepal_length', 'petal_length'])
kind = pn.widgets.Select(name='kind', value='scatter', options=['bivariate', 'scatter'])

plot = flowers.hvplot(x=x, y=y, kind=kind, colorbar=False, width=600)
pn.Row(pn.WidgetBox(x, y, kind), plot)

### Using functions

In addition to using widgets directly as arguments, we can also use functions that have been decorated with ``pn.depends``

In [ ]:
x = pn.widgets.Select(name='x', options=['sepal_width', 'petal_width'])
y = pn.widgets.Select(name='y', options=['sepal_length', 'petal_length'])
kind = pn.widgets.Select(name='kind', value='scatter', options=['bivariate', 'scatter'])
by_species = pn.widgets.Checkbox(name='By species')
color = pn.widgets.ColorPicker(value='#ff0000')

@pn.depends(by_species, color)
def by_species_fn(by_species, color):
    return 'species' if by_species else color

plot = flowers.hvplot(x=x, y=y, kind=kind, c=by_species_fn, colorbar=False, width=600, legend='top_right')

pn.Row(pn.WidgetBox(x, y, kind, color, by_species), plot)

We can keep even add a callback to disable the color options when 'bivariate' is selected. After running the cell below, try changing 'kind' above and notice how the color and 'By species' areas turn grey to indicate that they are disabled.

In [ ]:
def update(event):
    if kind.value == 'bivariate':
        color.disabled = True
        by_species.disabled = True
    else:
        color.disabled = False
        by_species.disabled = False

kind.param.watch(update, 'value');

To learn more about Panel and how to use it with output from hvPlot, see the [Panel docs on the HoloViews pane](https://panel.pyviz.org/reference/panes/HoloViews.html). To learn more about available widgets, see the Widgets' section of the [Panel Reference Gallery](https://panel.pyviz.org/reference/index.html).